# Predicting Orange Futures Prices Based on Weather

## CDS Algo Project S21

### Authors: Alex Lin, Brian Bobby, Divya Damodaran, Liam Bodurtha

### TODO for next week (updated 3/27)
- make table of production by county from map (https://www.nass.usda.gov/Statistics_by_State/Florida/Publications/Citrus/Citrus_Statistics/2015-16/fcs1516.pdf)
- Liam and Divya meeting this week to filter/clean data
    - talk about how to replace -99.00 values
    - decide on starting date
    - maybe make one massive dataframe
- Brian and Alex continue to research methodology and come up with concrete options for analysis

In [55]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import yfinance as yf
from datetime import date
import finplot as fplt
import requests
import pandas as pd
import numpy as np
import io
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [56]:
r1 = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_OJ1.csv?api_key=9BZrqXiy4Mck8_br9oG9')
futures_data_1 = pd.read_csv(io.StringIO(r1.text))

r2 = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_OJ2.csv?api_key=9BZrqXiy4Mck8_br9oG9')
futures_data_2 = pd.read_csv(io.StringIO(r2.text))

r3 = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_OJ3.csv?api_key=9BZrqXiy4Mck8_br9oG9')
futures_data_3 = pd.read_csv(io.StringIO(r3.text))

r4 = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_OJ4.csv?api_key=9BZrqXiy4Mck8_br9oG9')
futures_data_4 = pd.read_csv(io.StringIO(r4.text))

r5 = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_OJ5.csv?api_key=9BZrqXiy4Mck8_br9oG9')
futures_data_5 = pd.read_csv(io.StringIO(r5.text))

In [57]:
counties = ['Hendry', 'DeSoto', 'Polk', 'Highlands', 'Hardee', 'St Lucie', 'Collier', 'Indian River', 'Manatee', 'Charlotte', 'Lee','Lake', 'Osceola', 'Glades', 'Okeechobee', 'Hillsborough', 'Pasco', 'Martin', 'Orange', 'Brevard', 'Sarasota', 'Marion', 'Volusia', 'Hernando', 'Seminole']
boxes = [14282, 12773, 12539, 9735, 8436, 6693, 6677, 5965, 3282, 2696, 2128, 1606, 1463, 1360, 152, 718, 687, 684, 351, 335, 212, 163, 111, 91, 43]
productionDf = pd.DataFrame({'County:': counties, 'Boxes Produced of Oranges(in 1000s)': boxes })
productionDf

,County:,Boxes Produced of Oranges(in 1000s)
0,Hendry,14282
1,DeSoto,12773
2,Polk,12539
3,Highlands,9735
4,Hardee,8436
5,St Lucie,6693
6,Collier,6677
7,Indian River,5965
8,Manatee,3282
9,Charlotte,2696


In [58]:
futures_data_2.head()

,Date,Open,High,Low,Settle,Change,Wave,Volume,Prev. Day Open Interest,EFP Volume,EFS Volume,Block Volume
0,2021-04-16,116.0,117.05,115.8,116.20,0.05,116.54,921.0,6090.0,20.0,NaN,NaN
1,2021-04-15,114.0,116.50,113.9,116.15,2.60,115.28,826.0,5502.0,20.0,NaN,NaN
2,2021-04-14,112.6,114.15,112.0,113.55,0.80,113.14,604.0,5046.0,NaN,NaN,NaN
3,2021-04-13,112.0,112.70,112.0,112.75,0.65,112.13,608.0,4539.0,NaN,NaN,NaN
4,2021-04-12,113.1,113.10,112.0,112.10,-1.05,112.25,413.0,4200.0,NaN,NaN,NaN


In [59]:
futures_data_1['Date'] = pd.to_datetime(futures_data_1['Date'])
futures_data_2['Date'] = pd.to_datetime(futures_data_2['Date'])
futures_data_3['Date'] = pd.to_datetime(futures_data_3['Date'])
futures_data_4['Date'] = pd.to_datetime(futures_data_4['Date'])
futures_data_5['Date'] = pd.to_datetime(futures_data_5['Date'])

futures_data_1['Settle_1'] = futures_data_1['Settle']
futures_data_2['Settle_2'] = futures_data_2['Settle']
futures_data_3['Settle_3'] = futures_data_3['Settle']
futures_data_4['Settle_4'] = futures_data_4['Settle']
futures_data_5['Settle_5'] = futures_data_5['Settle']
futures_data_5.head()

,Date,Open,High,Low,Settle,Change,Wave,Volume,Prev. Day Open Interest,EFP Volume,EFS Volume,Block Volume,Settle_5
0,2021-04-16,124.50,124.5,124.50,123.90,-0.50,124.50,1.0,101.0,NaN,NaN,NaN,123.90
1,2021-04-15,122.95,123.0,122.95,124.40,2.05,122.98,8.0,104.0,NaN,NaN,NaN,124.40
2,2021-04-14,122.50,122.5,122.50,122.35,0.60,122.50,2.0,104.0,NaN,NaN,NaN,122.35
3,2021-04-13,NaN,NaN,NaN,121.75,0.60,NaN,NaN,104.0,NaN,NaN,NaN,121.75
4,2021-04-12,121.50,121.5,121.35,121.15,-1.00,121.49,11.0,93.0,NaN,NaN,NaN,121.15


In [60]:
futures_data_5.dtypes

Date                       datetime64[ns]
Open                              float64
High                              float64
Low                               float64
Settle                            float64
Change                            float64
Wave                              float64
Volume                            float64
Prev. Day Open Interest           float64
EFP Volume                        float64
EFS Volume                        float64
Block Volume                      float64
Settle_5                          float64
dtype: object

In [61]:
station_county = pd.read_csv('station_county_matching.csv')

In [62]:
station_county

,Station,County
0,APALACHICOLA AP,Franklin
1,ARCADIA,Desoto
2,ARCHBOLD BIO STN,Highlands
3,AVON PARK 2 W,Highlands
4,BARTOW,Polk
...,...,...
85,VERO BEACH 4SE,Indian River
86,WAUCHULA,Hardee
87,WEEKI WACHEE,Hernando
88,WEST PALM BEACH INTL AP,Palm Beach


In [63]:
weather_station_dfs = {}
for station in station_county['Station']:
    dfname = station.lower().replace(' ','_')
    weather_station_dfs[dfname] = pd.read_csv('orangeCsvs/'+station+'.csv')

In [64]:
weather_station_dfs['arcadia'].head()

,Unnamed: 0,id,year,month,day,precipitation,maxTemp,minTemp,meanTemp
0,0,80228,1899,1,1,-99.99,-99.9,-99.9,-99.90000
1,1,80228,1899,1,2,-99.99,-99.9,-99.9,-99.90000
2,2,80228,1899,1,3,-99.99,-99.9,-99.9,-99.90000
3,3,80228,1899,1,4,-99.99,-99.9,-99.9,-99.90000
4,4,80228,1899,1,5,-99.99,-99.9,-99.9,-99.90000


In [65]:
station_county[station_county['County'] == 'Collier']

,Station,County
37,IMMOKALEE,Collier
58,NAPLES,Collier
60,OASIS RS,Collier


In [66]:
stations_in_citrus_regions = []
for s in range(len(station_county.iloc[:,1].copy())):
    for c in counties:
        if c.lower().replace('saint', 'st') == station_county.iloc[s,1].lower():
              stations_in_citrus_regions.append(station_county.iloc[s,0].lower().replace(' ', '_'))
stations_in_citrus_regions.append('ft_pierce')
 #zero non-null rows - 3 more weather stations available for county 
print(stations_in_citrus_regions)

['arcadia', 'archbold_bio_stn', 'avon_park_2_w', 'bartow', 'bradenton_5_ese', 'brooksville_chin_hill', 'clermont_9_s', 'daytona_beach', 'daytona_beach_intl_ap', 'deland_1_sse', 'desoto_city_8_sw', 'devils_garden', 'fort_drum', 'ft_green_12_wsw', 'ft_myers_page_fld_ap', 'ft_pierce', 'hillsborough_river_sp', 'immokalee', 'kissimmee_2', 'la_belle', 'lisbon', 'melbourne_intl_ap', 'moore_haven_lock_1', 'mtn_lake', 'myakka_river_sp', 'naples', 'oasis_rs', 'ocala', 'okeechobee', 'orlando_intl_ap', 'parrish', 'plant_city', 'punta_gorda_4_ese', 'saint_leo', 'sanford', 'stuart', 'tampa_intl_ap', 'titusville', 'venice', 'vero_beach_intl_ap', 'vero_beach_4se', 'wauchula', 'weeki_wachee', 'ft_pierce']


In [67]:
# fil_weather_station_dfs contains the stations from counties that produce oranges
fil_weather_station_dfs = (weather_station_dfs.fromkeys(stations_in_citrus_regions))

for station in stations_in_citrus_regions:
    fil_weather_station_dfs[station] = weather_station_dfs[station].copy() 

Dropping:
* before start date of futures data (maybe 1967, feb2)
* replace -99.00's with NaN
* ~2 weather stations start at 1978 or later, a few more start between 1967 and 1978, so we should discuss trimming everything to 1978 and 

In [68]:
for dfname in stations_in_citrus_regions:
    fil_weather_station_dfs[dfname] = fil_weather_station_dfs[dfname][fil_weather_station_dfs[dfname]['year'] >= 1967]
    

In [69]:
print(len(fil_weather_station_dfs['oasis_rs']))
fil_weather_station_dfs['oasis_rs'].isnull().sum(axis = 0)

14637


Unnamed: 0       0
id               0
year             0
month            0
day              0
precipitation    0
maxTemp          0
minTemp          0
meanTemp         0
dtype: int64

In [70]:
#Making columns float types and dropping bad column
for station in stations_in_citrus_regions:
    df = fil_weather_station_dfs[station].copy()
    df['precipitation'] = df['precipitation'].astype(float)
    
    df['meanTemp'] = df['meanTemp'].replace(' ','-99.9')
    df['meanTemp'] = df['meanTemp'].astype('float64')
    
    fil_weather_station_dfs[station] = df

In [71]:
fil_weather_station_dfs['arcadia']

,Unnamed: 0,id,year,month,day,precipitation,maxTemp,minTemp,meanTemp
24888,24888,80228,1967,1,1,0.00,81.0,56.0,68.5
24889,24889,80228,1967,1,2,0.00,85.0,65.0,75.0
24890,24890,80228,1967,1,3,0.07,84.0,67.0,75.5
24891,24891,80228,1967,1,4,0.14,77.0,60.0,68.5
24892,24892,80228,1967,1,5,0.00,66.0,32.0,49.0
...,...,...,...,...,...,...,...,...,...
43546,43546,80228,2017,12,27,0.00,79.0,55.0,67.0
43547,43547,80228,2017,12,28,0.00,81.0,56.0,68.5
43548,43548,80228,2017,12,29,0.00,78.0,56.0,67.0
43549,43549,80228,2017,12,30,0.01,-99.9,50.0,-99.9


In [72]:
#replacing -99..9 with Nan
for station in stations_in_citrus_regions:
    fil_weather_station_dfs[station]['meanTemp'] = fil_weather_station_dfs[station]['meanTemp'].replace(-99.9, np.nan)
    
    fil_weather_station_dfs[station]['precipitation'] = fil_weather_station_dfs[station]['precipitation'].replace(-99.9, np.nan)
    
    fil_weather_station_dfs[station]['maxTemp'] = fil_weather_station_dfs[station]['maxTemp'].replace(-99.9, np.nan)
    
    fil_weather_station_dfs[station]['minTemp'] = fil_weather_station_dfs[station]['minTemp'].replace(-99.9, np.nan)


In [73]:
for station in stations_in_citrus_regions:
    fil_weather_station_dfs[station] = fil_weather_station_dfs[station].dropna()

In [74]:
#Dataframes that are only from counties that produce oranges with all year >= 1967 and rows with NaNs dropped
#we still have lots of data
fil_weather_station_dfs['arcadia'].isnull().sum(axis = 0)

Unnamed: 0       0
id               0
year             0
month            0
day              0
precipitation    0
maxTemp          0
minTemp          0
meanTemp         0
dtype: int64

In [75]:
#1970 - immokalee, 2010 - melbourne_intl_ap
for station in fil_weather_station_dfs.keys():
    print(station + ": " + str(len(fil_weather_station_dfs[station]['year'])))

arcadia: 17193
archbold_bio_stn: 17102
avon_park_2_w: 16668
bartow: 17317
bradenton_5_ese: 17267
brooksville_chin_hill: 15779
clermont_9_s: 17461
daytona_beach: 5005
daytona_beach_intl_ap: 17896
deland_1_sse: 16876
desoto_city_8_sw: 0
devils_garden: 15617
fort_drum: 13344
ft_green_12_wsw: 4862
ft_myers_page_fld_ap: 16994
ft_pierce: 17790
hillsborough_river_sp: 54
immokalee: 14594
kissimmee_2: 17107
la_belle: 14027
lisbon: 17552
melbourne_intl_ap: 2008
moore_haven_lock_1: 17760
mtn_lake: 17559
myakka_river_sp: 16715
naples: 17574
oasis_rs: 13227
ocala: 17675
okeechobee: 13338
orlando_intl_ap: 16736
parrish: 16696
plant_city: 17315
punta_gorda_4_ese: 17631
saint_leo: 17697
sanford: 17756
stuart: 16168
tampa_intl_ap: 17897
titusville: 16890
venice: 16311
vero_beach_intl_ap: 11686
vero_beach_4se: 17503
wauchula: 17417
weeki_wachee: 15215


In [76]:
for station in fil_weather_station_dfs.keys():
    if not station == 'desoto_city_8_sw':
        print(station + ": " + str(fil_weather_station_dfs[station]['year'].iloc[0]))

arcadia: 1967
archbold_bio_stn: 1969
avon_park_2_w: 1967
bartow: 1967
bradenton_5_ese: 1967
brooksville_chin_hill: 1967
clermont_9_s: 1967
daytona_beach: 2000
daytona_beach_intl_ap: 1967
deland_1_sse: 1967
devils_garden: 1967
fort_drum: 1967
ft_green_12_wsw: 2001
ft_myers_page_fld_ap: 1967
ft_pierce: 1967
hillsborough_river_sp: 1967
immokalee: 1970
kissimmee_2: 1967
la_belle: 1967
lisbon: 1967
melbourne_intl_ap: 2010
moore_haven_lock_1: 1967
mtn_lake: 1967
myakka_river_sp: 1968
naples: 1967
oasis_rs: 1978
ocala: 1967
okeechobee: 1967
orlando_intl_ap: 1967
parrish: 1967
plant_city: 1967
punta_gorda_4_ese: 1967
saint_leo: 1967
sanford: 1967
stuart: 1967
tampa_intl_ap: 1967
titusville: 1967
venice: 1967
vero_beach_intl_ap: 1984
vero_beach_4se: 1967
wauchula: 1967
weeki_wachee: 1969


In [77]:
#Try merging the station results for each county for each county. If a row is Nan in one station ignore and average witht the other 

In [78]:
station_county['Station'] = station_county['Station'].apply(lambda x: x.lower().replace(' ', '_'))
station_county['County'] = station_county['County'].apply(lambda x: x.lower().replace('saint', 'st'))

county_dfs = {}

for county in counties:
    if county == "St. Lucie":
        county = "St Lucie"
        
    county = county.lower()
    df = pd.DataFrame()
    
    for stat in station_county[station_county['County'] == county]['Station']:
        if stat in fil_weather_station_dfs.keys():
            
            temp_df = fil_weather_station_dfs[stat].copy()
            temp_df['County'] = county
        
            df = df.append(temp_df)
    
    county_dfs[county] = df


In [79]:
productionDf['County:'] = productionDf['County:'].apply(lambda x: x.lower().replace('st.', 'st'))

In [80]:
total_boxes = 0
for coun in county_dfs.keys():
    mult = int(productionDf[productionDf['County:'] == coun]['Boxes Produced of Oranges(in 1000s)'])
    total_boxes += mult
    county_dfs[coun] = (county_dfs[coun].groupby(['year', 'month', 'day']).mean())
    df = county_dfs[coun]
    df['minTemp'] = df['minTemp']*mult
    df['maxTemp'] = df['maxTemp']*mult
    df['meanTemp'] = df['meanTemp']*mult
    df['precipitation'] = df['precipitation']*mult

    df['weight'] = mult
    county_dfs[coun] = df
    print(len(df))

17191
17193
17866
17816
17534
17790
17890
17877
17824
17631
16994
17890
17107
17760
15788
17897
17697
16168
16736
17004
17761
17675
17897
17027
17756


In [81]:
len(county_dfs.keys())

25

In [82]:
final_df = pd.DataFrame()
for county in county_dfs.keys():
    final_df = final_df.append(county_dfs[county])
    
final_df = final_df.groupby(['year', 'month', 'day']).sum()


In [83]:
print(final_df)

                Unnamed: 0         id  precipitation    maxTemp    minTemp  \
year month day                                                               
1967 1     1      356667.0  2046299.5          0.000  7630978.0  5356702.0   
           2      356691.0  2046299.5        164.010  7791664.5  5962311.0   
           3      356715.0  2046299.5       5021.840  7838329.0  6137728.5   
           4      346854.0  1961674.5      25021.380  7084575.0  5353286.0   
           5      356763.0  2046299.5       4167.660  5982874.0  3467413.5   
...                    ...        ...            ...        ...        ...   
2017 12    27     741149.0  1963113.0          0.000  7313722.0  5217165.5   
           28     735865.0  1963088.0          0.555  7460119.5  5385999.0   
           29     735888.0  1963088.0         60.035  7101182.0  5244295.0   
           30     697669.0  1882885.0          1.630  5640480.0  4135126.0   
           31     675003.0  1794903.0          0.000  5590215.0 

In [84]:
weighted_df = final_df.copy()
weighted_df['minTemp'] = weighted_df['minTemp']/weighted_df['weight']
weighted_df['maxTemp'] = weighted_df['maxTemp']/weighted_df['weight']
weighted_df['meanTemp'] = weighted_df['meanTemp']/weighted_df['weight']
weighted_df['precipitation'] = weighted_df['precipitation']/weighted_df['weight']

In [85]:
weighted_df['weight'] = weighted_df['weight']/total_boxes

In [86]:
weighted_df = weighted_df.reset_index().drop(columns = 'id')
weighted_df

,year,month,day,Unnamed: 0,precipitation,maxTemp,minTemp,meanTemp,weight
0,1967,1,1,356667.0,0.000000,82.188741,57.693862,69.941301,0.996405
1,1967,1,2,356691.0,0.001766,83.919400,64.216517,74.067959,0.996405
2,1967,1,3,356715.0,0.054087,84.421995,66.105835,75.263915,0.996405
3,1967,1,4,346854.0,0.273805,77.525333,58.580123,68.052728,0.980704
4,1967,1,5,356763.0,0.044887,64.437989,37.345455,50.891722,0.996405
...,...,...,...,...,...,...,...,...,...
17892,2017,12,27,741149.0,0.000000,78.693788,56.135374,67.414581,0.997392
17893,2017,12,28,735865.0,0.000006,80.268988,57.951979,69.110484,0.997392
17894,2017,12,29,735888.0,0.000646,76.406912,56.427280,66.417096,0.997392
17895,2017,12,30,697669.0,0.000020,70.360003,51.582042,60.971023,0.860316


In [87]:
weighted_df['Date'] = pd.to_datetime(weighted_df[['year', 'month', 'day']])
weighted_df = weighted_df.drop(columns = ['year', 'month', 'day'])
weighted_df.head()

,Unnamed: 0,precipitation,maxTemp,minTemp,meanTemp,weight,Date
0,356667.0,0.000000,82.188741,57.693862,69.941301,0.996405,1967-01-01
1,356691.0,0.001766,83.919400,64.216517,74.067959,0.996405,1967-01-02
2,356715.0,0.054087,84.421995,66.105835,75.263915,0.996405,1967-01-03
3,346854.0,0.273805,77.525333,58.580123,68.052728,0.980704,1967-01-04
4,356763.0,0.044887,64.437989,37.345455,50.891722,0.996405,1967-01-05


In [88]:
weighted_df.dtypes

Unnamed: 0              float64
precipitation           float64
maxTemp                 float64
minTemp                 float64
meanTemp                float64
weight                  float64
Date             datetime64[ns]
dtype: object

In [89]:
futures_and_weather_df = pd.merge(futures_data_1, weighted_df, how="inner", on="Date")
futures_and_weather_df = pd.merge(futures_data_2, futures_and_weather_df, how="inner", on="Date")
futures_and_weather_df = pd.merge(futures_data_3, futures_and_weather_df, how="inner", on="Date")
futures_and_weather_df = pd.merge(futures_data_4, futures_and_weather_df, how="inner", on="Date")
futures_and_weather_df = pd.merge(futures_data_5, futures_and_weather_df, how="inner", on="Date")
futures_and_weather_df.head()

,Date,Open,High,Low,Settle,Change,Wave,Volume,Prev. Day Open Interest,EFP Volume,...,EFP Volume_y,EFS Volume_y,Block Volume_y,Settle_1,Unnamed: 0,precipitation,maxTemp,minTemp,meanTemp,weight
0,2017-12-29,137.55,137.55,137.55,140.15,0.75,137.55,1.0,69.0,NaN,...,NaN,NaN,NaN,136.00,735888.0,0.000646,76.406912,56.427280,66.417096,0.997392
1,2017-12-28,NaN,NaN,NaN,139.40,3.05,NaN,NaN,69.0,NaN,...,NaN,NaN,NaN,136.70,735865.0,0.000006,80.268988,57.951979,69.110484,0.997392
2,2017-12-27,136.50,136.65,136.10,136.35,-0.35,136.52,13.0,64.0,NaN,...,52.0,NaN,NaN,133.15,741149.0,0.000000,78.693788,56.135374,67.414581,0.997392
3,2017-12-26,137.30,137.30,137.30,136.70,NaN,137.30,2.0,63.0,NaN,...,NaN,NaN,NaN,134.05,654768.0,0.000000,77.516503,54.505437,66.010970,0.788489
4,2017-12-22,NaN,NaN,NaN,138.55,-0.30,NaN,NaN,63.0,NaN,...,NaN,NaN,NaN,135.25,741034.0,0.003400,81.792918,61.542716,71.667817,0.997392


In [90]:
futures_and_weather_df = futures_and_weather_df[['Date', 'Settle_1', 'Settle_2', 'Settle_3', 'Settle_4', 'Settle_5', 'precipitation', 'minTemp', 'meanTemp', 'weight']]
futures_and_weather_df

,Date,Settle_1,Settle_2,Settle_3,Settle_4,Settle_5,precipitation,minTemp,meanTemp,weight
0,2017-12-29,136.00,136.85,137.80,139.65,140.15,0.000646,56.427280,66.417096,0.997392
1,2017-12-28,136.70,135.85,136.85,138.65,139.40,0.000006,57.951979,69.110484,0.997392
2,2017-12-27,133.15,132.35,133.75,135.45,136.35,0.000000,56.135374,67.414581,0.997392
3,2017-12-26,134.05,133.85,135.15,136.35,136.70,0.000000,54.505437,66.010970,0.788489
4,2017-12-22,135.25,135.90,136.95,138.00,138.55,0.003400,61.542716,71.667817,0.997392
...,...,...,...,...,...,...,...,...,...,...
8511,1967-07-06,38.55,40.05,41.25,42.35,36.75,0.300590,72.102694,81.237543,0.895527
8512,1967-07-05,38.25,40.00,41.40,42.20,36.65,0.230177,71.605480,80.951361,0.895527
8513,1967-05-31,41.00,42.50,43.15,42.50,38.30,0.130251,65.606643,80.191397,0.895527
8514,1967-05-29,41.70,42.80,43.45,42.80,39.00,0.003383,68.352625,81.427415,0.895527


## Model Training Starts Here

In [ ]:
temp = weighted_df["meanTemp"]
rain = weighted_df["precipitation"]
price = futures_data["Settle"]

In [ ]:
linreg = LinearRegression().fit(X, y) #linear reg
logreg = LogisticRegression().fit(x,y)

In [ ]:
#train_test_split on temp rain and price
#best if done on df contain all 3 the separating them into 3

In [ ]:
#Cross Validation:
#for i in range(5):